In [1]:
import torch
import cv2
import time
import pytesseract
import re
import numpy as np
import easyocr

C:\Users\DINESHKUMAR S\anaconda3\envs\DeepLearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EASY_OCR = easyocr.Reader(['en'])
OCR_TH = 0.1

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
def detectx (frame, model):
    frame = [frame]
    print(f"[INFO] Detecting. . . ")
    results = model(frame)

    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, cordinates

In [4]:
def plot_boxes(results, frame,classes):
    
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    print(f"[INFO] Total {n} detections. . . ")
    print(f"[INFO] Looping through all detections. . . ")


    for i in range(n):
        row = cord[i]
        if row[4] >= 0.30:
            print(f"[INFO] Extracting BBox coordinates. . . ")
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            text_d = classes[int(labels[i])]

            coords = [x1,y1,x2,y2]

            plate_num = recognize_plate_easyocr(img = frame, coords= coords, reader= EASY_OCR, region_threshold= OCR_TH)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255,0), -1)
            cv2.putText(frame, f"{plate_num}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255), 2)

    return frame

In [5]:
def recognize_plate_easyocr(img, coords,reader,region_threshold):
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin)-5:int(ymax)+5, int(xmin)-5:int(xmax)+5]


    ocr_result = reader.readtext(nplate)

    text = filter_text(region=nplate, ocr_result=ocr_result, region_threshold= region_threshold)

    if len(text)>=1:
        for i in range(len(text)):
            text[i] = text[i].upper()
    return text

In [6]:
def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    print(ocr_result)
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size >=region_threshold:
            plate.append(result[1])
        for i in range(len(plate)):
            if(len(plate[i])>=8):
                if plate[i][2]!=" ":
                    plate[i]=plate[i][:2]+" "+plate[i][2:]
                if plate[i][5]!=" ":
                    plate[i]=plate[i][:5]+" "+plate[i][5:]
                if plate[i][8]!=" ":
                    plate[i]=plate[i][:8]+" "+plate[i][8:]
            plate[i]=plate[i].upper()
            print(plate[i]+"\n")
    return plate

In [7]:
def main(img_path=None, vid_path=None,vid_out = None):

    print(f"[INFO] Loading model... ")
    model =  torch.hub.load("./yolov5", 'custom', source ='local', path='last.pt',force_reload=True) 

    classes = model.names

    if img_path != None:
        print(f"[INFO] Working with image: {img_path}")
        img_out_name = f"./output/result_{img_path.split('/')[-1]}"

        frame = cv2.imread(img_path)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        results = detectx(frame, model = model)   

        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

        frame = plot_boxes(results, frame,classes = classes)
        
        cv2.namedWindow("img_only", cv2.WINDOW_NORMAL)

        while True:
            cv2.imshow("img_only", frame)
            if cv2.waitKey(5) & 0xFF == ord('q'):
                print(f"[INFO] Exiting. . . ")
                cv2.imwrite(f"{img_out_name}",frame)
                break
    cv2.destroyAllWindows()
    

    if vid_path != None:
        print(f"[INFO] Working with video: {vid_path}")

        cap = cv2.VideoCapture(vid_path)

        if vid_out:

            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            codec=cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(vid_out, codec, fps, (width, height))

        assert cap.isOpened()
        frame_no = 1

        cv2.namedWindow("vid_out", cv2.WINDOW_NORMAL)
        while True:
            ret, frame = cap.read()
            if ret  and frame_no %1 == 0:
                print(f"[INFO] Working with frame {frame_no} ")

                frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                results = detectx(frame, model = model)
                frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)


                frame = plot_boxes(results, frame,classes = classes)
                
                cv2.imshow("vid_out", frame)
                if vid_out:
                    print(f"[INFO] Saving output video. . . ")
                    out.write(frame)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                frame_no += 1
        
        print(f"[INFO] Clening up. . . ")
        out.release()
        
    cv2.destroyAllWindows()
    

In [8]:
main(vid_path=r"C:\Users\DINESHKUMAR S\Downloads\PXL_20220913_080309491_AdobeExpress (1).mp4",vid_out="PXL_20220913_080309491.mp4") 
#main(vid_path=0,vid_out="webcam_facemask_result.mp4")
#main(img_path=r"E:\License Plate Number recognition\train\images\frame-25.jpg") 

[INFO] Loading model... 


YOLOv5  2022-11-13 Python-3.7.15 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


[INFO] Working with video: C:\Users\DINESHKUMAR S\Downloads\PXL_20220913_080309491_AdobeExpress (1).mp4
[INFO] Working with frame 1 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Saving output video. . . 
[INFO] Working with frame 2 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Saving output video. . . 
[INFO] Working with frame 3 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Saving output video. . . 
[INFO] Working with frame 4 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Saving output video. . . 
[INFO] Working with frame 5 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Saving output video. . . 
[INFO] Working with frame 6 
[INFO] Detecting. . . 
[INFO] Total 0 detections. . . 
[INFO] 